## Quantum State Tomography (QST) using Maximum Likelihood Estimation (MLE)

### Installing python packages

In [ ]:
!pip install qutip
!pip install ipywidgets
!pip install requests

### Importing python packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import ipywidgets as widgets
from IPython.display import display, clear_output
from qutip import Bloch
import matplotlib.patches as patches
from ipywidgets import interactive_output
from matplotlib.ticker import MultipleLocator, FuncFormatter
import requests

### MLE

In [ ]:
def create_state(theta, phi):
    """Create a normalized qubit state."""
    return np.array([np.cos(theta/2), np.exp(1j * phi) * np.sin(theta/2)])

def measurement_probability(state, basis):
    """Calculate probability of measuring |0⟩ in the given basis."""
    theta = np.real(2*np.acos(state[0]))
    phi = np.angle(state[1])
    if basis == 'Z':
        return (1 + np.cos(theta))/2
    elif basis == 'X':
        return (1 + np.cos(phi) * np.sin(theta)) / 2
    elif basis == 'Y':
        return (1 + np.sin(phi) * np.sin(theta)) / 2

def simulate_measurements(state, basis, n_shots):
    """Simulate measurements in the given basis."""
    prob_plus1 = measurement_probability(state, basis)
    return np.random.choice([+1, -1], size=n_shots, p=[prob_plus1, 1-prob_plus1])

def log_likelihood(params, measurements_x, measurements_y, measurements_z):
    """Negative log-likelihood function."""
    theta, phi = params
    state = create_state(theta, phi)

    log_L = 0
    for basis, measurements in zip(['X', 'Y', 'Z'], [measurements_x, measurements_y, measurements_z]):
        prob_plus1 = measurement_probability(state, basis)
        prob_minus1 = 1 - prob_plus1
        n_plus1 = np.sum(measurements == +1)
        n_minus1 = np.sum(measurements == -1)
        log_L += n_plus1 * np.log(prob_plus1 + 1e-10) + n_minus1 * np.log(prob_minus1 + 1e-10)

    return -log_L

def estimate_parameters(measurements_x, measurements_y, measurements_z):
    """Estimate θ and φ using MLE."""
    opt_val = []
    opt_params = []
    for _ in range(50):
      theta_init = np.random.uniform(0, np.pi)
      phi_init = np.random.uniform(0, 2*np.pi)
      initial_params = [theta_init, phi_init]
      result = minimize(log_likelihood, initial_params, args=(measurements_x, measurements_y, measurements_z),
                      bounds=[(0, np.pi), (0, 2*np.pi-0.01)], method='L-BFGS-B')
      opt_val.append(result.fun)
      opt_params.append(result.x)

    result = min(opt_val)
    index = opt_val.index(result)
    result = opt_params[index]
    return result

def fidelity(state1, state2):
    """Compute fidelity between two quantum states."""
    return np.abs(np.vdot(state1, state2))**2

def plot_bloch_sphere(ax, theta, phi, title):
    """Plot Bloch sphere representation on given axes."""
    b = Bloch(axes=ax)
    b.vector_color = ['r']
    b.add_vectors([np.sin(theta) * np.cos(phi), np.sin(theta) * np.sin(phi), np.cos(theta)])
    b.render()
    ax.set_title(title)

### Interactive Tool Code

In [ ]:
# Main interactive update function
def update_plot_degrees(theta_deg, phi_deg, n_shots_x, n_shots_y, n_shots_z):
    theta = np.radians(theta_deg)
    phi = np.radians(phi_deg)
    update_plot(theta, phi, n_shots_x, n_shots_y, n_shots_z)

def update_plot(theta, phi, n_shots_x, n_shots_y, n_shots_z):
    clear_output(wait=True)

    true_state = create_state(theta, phi)
    measurements_x = simulate_measurements(true_state, 'X', n_shots_x)
    measurements_y = simulate_measurements(true_state, 'Y', n_shots_y)
    measurements_z = simulate_measurements(true_state, 'Z', n_shots_z)

    est_theta, est_phi = estimate_parameters(measurements_x, measurements_y, measurements_z)
    reconstructed_state = create_state(est_theta, est_phi)
    fid = fidelity(true_state, reconstructed_state)

    fig = plt.figure(figsize=(14, 5))
    gs_main = fig.add_gridspec(1, 4, width_ratios=[1, 1, 0.05, 1.2], wspace=0.3)

    ax1 = fig.add_subplot(gs_main[0], projection='3d')
    plot_bloch_sphere(ax1, theta, phi, '')
    ax1.set_title("True State", pad=20, fontsize=15)

    ax2 = fig.add_subplot(gs_main[1], projection='3d')
    plot_bloch_sphere(ax2, est_theta, est_phi, '')
    ax2.set_title("Reconstructed State", pad=20, fontsize=15)

    gs = gs_main[3].subgridspec(2, 1, height_ratios=[1, 3], hspace=0.3)

    ax_fid = fig.add_subplot(gs[0])
    ax_fid.set_xlim(0, 1)
    ax_fid.set_ylim(0, 1)
    ax_fid.axis('off')
    bg_bar = patches.Rectangle((0, 0.4), 1, 0.2, color='lightgray', ec='lightgray')
    fid_bar = patches.Rectangle((0, 0.4), fid, 0.2, color='red')
    ax_fid.add_patch(bg_bar)
    ax_fid.add_patch(fid_bar)
    ax_fid.text(0.5, 0.8, f"Fidelity: {fid:.4f}", ha='center', fontsize=15)

    ax_bar = fig.add_subplot(gs[1])
    labels = [r'$\theta_{\text{True}}$', r'$\theta_{\text{Est}}$', r'$\phi_{\text{True}}$', r'$\phi_{\text{Est}}$']
    values = [theta, est_theta, phi, est_phi]
    colors = ['green', 'lightgreen', 'blue', 'lightblue']
    x_pos = np.arange(len(labels))
    ax_bar.bar(x_pos, values, color=colors)
    ax_bar.set_xticks(x_pos)
    ax_bar.set_xticklabels(labels)
    ax_bar.set_ylim(0, 2 * np.pi + 0.75)

    def pi_formatter(x, pos):
        frac = x / np.pi
        if np.isclose(frac, 0): return "0"
        elif np.isclose(frac, 1): return r"$\pi$"
        elif np.isclose(frac, 2): return r"$2\pi$"
        elif np.isclose(frac, 0.5): return r"$\pi/2$"
        elif np.isclose(frac, 1.5): return r"$3\pi/2$"
        else: return f"{frac:.1f}π"

    ax_bar.yaxis.set_major_locator(MultipleLocator(np.pi / 2))
    ax_bar.yaxis.set_major_formatter(FuncFormatter(pi_formatter))
    ax_bar.set_ylabel("Angle (rad)")
    ax_bar.set_title('Bloch Sphere parameters')

    for i, val in enumerate(values):
        deg_val = np.degrees(val)
        ax_bar.text(i, val + 0.15, f"{deg_val:.1f}°", ha='center', va='bottom', fontsize=12)

    fig.subplots_adjust(left=0.06, right=0.97, top=0.92, bottom=0.15)
    plt.show()

# Sliders
def styled_slider(slider, label):
    return widgets.VBox([
        widgets.HTML(f"<span style='font-size:18px; font-weight:600'>{label}</span>"),
        slider
    ])

theta_deg_slider = widgets.FloatSlider(min=0, max=180, step=1, value=45, layout=widgets.Layout(width='400px', height='40px'))
phi_deg_slider = widgets.FloatSlider(min=0, max=360, step=1, value=45, layout=widgets.Layout(width='400px', height='40px'))
n_shots_x_slider = widgets.IntSlider(min=0, max=10000, step=100, value=0, layout=widgets.Layout(width='400px', height='40px'))
n_shots_y_slider = widgets.IntSlider(min=0, max=10000, step=100, value=0, layout=widgets.Layout(width='400px', height='40px'))
n_shots_z_slider = widgets.IntSlider(min=0, max=10000, step=100, value=1000, layout=widgets.Layout(width='400px', height='40px'))

angle_sliders = widgets.VBox([
    styled_slider(theta_deg_slider, "θ (°)"),
    styled_slider(phi_deg_slider, "φ (°)")
])

shot_sliders = widgets.VBox([
    styled_slider(n_shots_x_slider, "X Shots"),
    styled_slider(n_shots_y_slider, "Y Shots"),
    styled_slider(n_shots_z_slider, "Z Shots")
])

# Create Bloch sphere image
bloch_image = widgets.Image(
    # value=requests.get("https://www.datocms-assets.com/119587/1534934969-blochspphere.jpg").content,
    value=requests.get("https://raw.githubusercontent.com/mdaamirQ/QubitLens/refs/heads/main/BlochSphere.png").content,
    format='png',
    width=320,
    height=320
)

# Final layout
slider_row = widgets.HBox([
    bloch_image,
    widgets.HTML("<div style='width:40px'></div>"),  # Spacer
    shot_sliders,
    widgets.HTML("<div style='width:40px'></div>"),
    angle_sliders
])

interactive_plot = interactive_output(update_plot_degrees, {
    'theta_deg': theta_deg_slider,
    'phi_deg': phi_deg_slider,
    'n_shots_x': n_shots_x_slider,
    'n_shots_y': n_shots_y_slider,
    'n_shots_z': n_shots_z_slider
})

## QubitLens

In [ ]:
display(interactive_plot, slider_row)

## Using Pennylane

In [ ]:
!pip install pennylane
!pip install qutip

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from qutip import Bloch
import matplotlib.patches as patches
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import interactive_output
from matplotlib.ticker import MultipleLocator, FuncFormatter
from scipy.optimize import minimize


dev = qml.device("default.qubit", wires=1, shots=None)

def circuit(theta, phi):
    qml.RY(theta, wires=0)
    qml.RZ(phi, wires=0)

@qml.qnode(dev)
def prob_z(theta, phi):
    circuit(theta, phi)
    return qml.probs(wires=0)

@qml.qnode(dev)
def prob_x(theta, phi):
    circuit(theta, phi)
    qml.Hadamard(wires=0)
    return qml.probs(wires=0)

@qml.qnode(dev)
def prob_y(theta, phi):
    circuit(theta, phi)
    qml.RX(-np.pi/2, wires=0)
    return qml.probs(wires=0)


def simulate_measurements(theta, phi, basis, shots=1000):
    if basis == 'X':
        probs = prob_x(theta, phi)
    elif basis == 'Y':
        probs = prob_y(theta, phi)
    elif basis == 'Z':
        probs = prob_z(theta, phi)
    return np.random.choice([0, 1], size=shots, p=probs)

def log_likelihood(params, mx, my, mz):
    theta, phi = params
    loss = 0
    for basis, m in zip(['X', 'Y', 'Z'], [mx, my, mz]):
        if basis == 'X':
            p0 = prob_x(theta, phi)[0]
        elif basis == 'Y':
            p0 = prob_y(theta, phi)[0]
        elif basis == 'Z':
            p0 = prob_z(theta, phi)[0]
        n0 = np.sum(m == 0)
        n1 = np.sum(m == 1)
        loss -= n0 * np.log(p0 + 1e-10) + n1 * np.log(1 - p0 + 1e-10)
    return loss

def estimate_parameters(mx, my, mz):
    best = float('inf')
    best_params = None
    for _ in range(30):
        init = [np.random.uniform(0, np.pi), np.random.uniform(0, 2*np.pi)]
        res = minimize(log_likelihood, init, args=(mx, my, mz), bounds=[(0, np.pi), (0, 2*np.pi)])
        if res.fun < best:
            best = res.fun
            best_params = res.x
    return best_params

def create_state(theta, phi):
    return np.array([np.cos(theta/2), np.exp(1j*phi) * np.sin(theta/2)])

def fidelity(state1, state2):
    return np.abs(np.vdot(state1, state2))**2

def plot_bloch_sphere(ax, theta, phi, title):
    b = Bloch(axes=ax)
    b.vector_color = ['r']
    b.add_vectors([np.sin(theta) * np.cos(phi), np.sin(theta) * np.sin(phi), np.cos(theta)])
    b.render()
    ax.set_title(title, pad=20, fontsize=15)

def update_plot_degrees(theta_deg, phi_deg, shots_x, shots_y, shots_z):
    theta = np.radians(theta_deg)
    phi = np.radians(phi_deg)
    update_plot(theta, phi, shots_x, shots_y, shots_z)

def update_plot(theta, phi, shots_x, shots_y, shots_z):
    clear_output(wait=True)

    true_state = create_state(theta, phi)
    mx = simulate_measurements(theta, phi, 'X', shots_x)
    my = simulate_measurements(theta, phi, 'Y', shots_y)
    mz = simulate_measurements(theta, phi, 'Z', shots_z)

    est_theta, est_phi = estimate_parameters(mx, my, mz)
    est_state = create_state(est_theta, est_phi)
    fid = fidelity(true_state, est_state)

    fig = plt.figure(figsize=(14, 5))
    gs_main = fig.add_gridspec(1, 4, width_ratios=[1, 1, 0.05, 1.2], wspace=0.3)

    ax1 = fig.add_subplot(gs_main[0], projection='3d')
    plot_bloch_sphere(ax1, theta, phi, "True State")

    ax2 = fig.add_subplot(gs_main[1], projection='3d')
    plot_bloch_sphere(ax2, est_theta, est_phi, "Reconstructed State")

    gs = gs_main[3].subgridspec(2, 1, height_ratios=[1, 3], hspace=0.3)

    ax_fid = fig.add_subplot(gs[0])
    ax_fid.set_xlim(0, 1)
    ax_fid.set_ylim(0, 1)
    ax_fid.axis('off')
    bg_bar = patches.Rectangle((0, 0.4), 1, 0.2, color='lightgray', ec='lightgray')
    fid_bar = patches.Rectangle((0, 0.4), fid, 0.2, color='red')
    ax_fid.add_patch(bg_bar)
    ax_fid.add_patch(fid_bar)
    ax_fid.text(0.5, 0.8, f"Fidelity: {fid:.4f}", ha='center', fontsize=15)

    ax_bar = fig.add_subplot(gs[1])
    labels = [r'$\theta_{\text{True}}$', r'$\theta_{\text{Est}}$', r'$\phi_{\text{True}}$', r'$\phi_{\text{Est}}$']
    values = [theta, est_theta, phi, est_phi]
    colors = ['green', 'lightgreen', 'blue', 'lightblue']
    x_pos = np.arange(len(labels))
    ax_bar.bar(x_pos, values, color=colors)
    ax_bar.set_xticks(x_pos)
    ax_bar.set_xticklabels(labels)
    ax_bar.set_ylim(0, 2*np.pi + 0.75)

    def pi_formatter(x, pos):
        frac = x / np.pi
        if np.isclose(frac, 0): return "0"
        elif np.isclose(frac, 1): return r"$\pi$"
        elif np.isclose(frac, 2): return r"$2\pi$"
        elif np.isclose(frac, 0.5): return r"$\pi/2$"
        elif np.isclose(frac, 1.5): return r"$3\pi/2$"
        else: return f"{frac:.1f}π"

    ax_bar.yaxis.set_major_locator(MultipleLocator(np.pi/2))
    ax_bar.yaxis.set_major_formatter(FuncFormatter(pi_formatter))
    ax_bar.set_ylabel("Angle (rad)", fontsize=13)
    ax_bar.set_title("Bloch Sphere Parameters", fontsize=15)
    ax_bar.tick_params(axis='x', labelsize=13)
    ax_bar.tick_params(axis='y', labelsize=13)

    for i, val in enumerate(values):
        ax_bar.text(i, val + 0.15, f"{np.degrees(val):.1f}°", ha='center', fontsize=12)

    fig.subplots_adjust(left=0.06, right=0.97, top=0.92, bottom=0.15)
    plt.show()

# Interactive widgets
def styled_slider(slider, label):
    return widgets.VBox([
        widgets.HTML(f"<span style='font-size:18px; font-weight:600'>{label}</span>"),
        slider
    ])

theta_slider = widgets.FloatSlider(min=0, max=180, step=1, value=45,
                                   layout=widgets.Layout(width='400px', height='40px'))
phi_slider = widgets.FloatSlider(min=0, max=360, step=1, value=45,
                                 layout=widgets.Layout(width='400px', height='40px'))

x_slider = widgets.IntSlider(min=0, max=10000, step=100, value=0,
                             layout=widgets.Layout(width='400px', height='40px'))
y_slider = widgets.IntSlider(min=0, max=10000, step=100, value=0,
                             layout=widgets.Layout(width='400px', height='40px'))
z_slider = widgets.IntSlider(min=0, max=10000, step=100, value=1000,
                             layout=widgets.Layout(width='400px', height='40px'))

angle_sliders = widgets.VBox([
    styled_slider(theta_slider, "θ (°)"),
    styled_slider(phi_slider, "φ (°)")
])

shot_sliders = widgets.VBox([
    styled_slider(x_slider, "X Shots"),
    styled_slider(y_slider, "Y Shots"),
    styled_slider(z_slider, "Z Shots")
])

slider_row = widgets.HBox([
    shot_sliders,
    widgets.HTML("<div style='width:40px'></div>"),
    angle_sliders
])

interactive_plot = interactive_output(update_plot_degrees, {
    'theta_deg': theta_slider,
    'phi_deg': phi_slider,
    'shots_x': x_slider,
    'shots_y': y_slider,
    'shots_z': z_slider
})

In [ ]:
display(interactive_plot, slider_row)
